In [1]:
import pandas as pd
import random
from scipy.optimize import linear_sum_assignment

In [2]:
# number of Constraints, BNBs and Users
n = {'c': 4, 'b': 6, 'u': 4}

In [3]:
# the list of constraints for this session
C = [ f"c_{c}" for c in range(n['c']) ]
C

['c_0', 'c_1', 'c_2', 'c_3']

In [4]:
# the list of users for this session
users = [ f"u_{u}" for u in range(n['u']) ]
users

['u_0', 'u_1', 'u_2', 'u_3']

In [5]:
def generate_B(): 
    dt = { 
        f"b_{b}": [ random.choice([0, 1]) for c in C ] 
        for b in range(n['b']) 
    }
    return pd.DataFrame(dt, index=C).transpose()

def generate_U():
    dt = { 
        c: [ random.choice([0, 1]) for u in range(n['u']) ] 
        for c in C 
    }
    return pd.DataFrame(dt, index=users).transpose()

In [6]:
# generate the B matrix:
#   rows: BNBs
#   cols: constraints that the BNB does or does not meet
B = generate_B()
B

,c_0,c_1,c_2,c_3
b_0,1,1,0,0
b_1,0,1,1,1
b_2,1,0,0,1
b_3,0,0,1,1
b_4,0,1,1,1
b_5,0,1,1,0


In [7]:
# generate the U matrix:
#   rows: constraints
#   cols: user that may or may not require constraint
U = generate_U()
U

,u_0,u_1,u_2,u_3
c_0,0,0,0,1
c_1,1,1,0,0
c_2,0,1,1,0
c_3,0,1,1,0


In [8]:
# create A matrix - score for pairing User and BNB.
#   rows: BNB
#   cols: User 
A = B.dot(U)
A

,u_0,u_1,u_2,u_3
b_0,1,1,0,1
b_1,1,3,2,0
b_2,0,1,1,1
b_3,0,2,2,0
b_4,1,3,2,0
b_5,1,2,1,0


In [9]:
# define the cost matrix: apply cell=n_c-cell for each cell in A
cost = A.copy().to_numpy()
An = A.to_numpy()
mx_cost = len(C)
for row in cost:
    for u in range(len(users)):
        row[u] = mx_cost - row[u]
cost

array([[3, 3, 4, 3],
       [3, 1, 2, 4],
       [4, 3, 3, 3],
       [4, 2, 2, 4],
       [3, 1, 2, 4],
       [3, 2, 3, 4]])

In [10]:
# apply CAP to cost matrix to get indexes denoting the BNB and User pairings
bs, us = linear_sum_assignment(cost)

In [15]:
# report the outcome of the algorithm
user2bnb = {} # pairing of each user to their bnb
user2satisfaction = {} 
dt = {}
for ri, row in enumerate(A.iterrows()):
    row_name = row[0]
    new_row = [ f" {col} " for col in row[1] ] 
    if ri in bs:
        b = list(bs).index(ri)
        u = us[b]
        new_row[u] = f"[{new_row[u][1:-1]}]"
        user2bnb[f"u_{u}"] = f"b_{b}"
        user2satisfaction[f"u_{u}"] = int(new_row[u][1:-1])
    dt[row_name] = new_row
print("Pairings:")
pd.DataFrame(dt, index=users).transpose()

Pairings:


,u_0,u_1,u_2,u_3
b_0,[1],1,0,1
b_1,1,[3],2,0
b_2,0,1,1,[1]
b_3,0,2,[2],0
b_4,1,3,2,0
b_5,1,2,1,0


In [12]:
# print statistics
stats = {
    'user': [],
    'bnb': [],
    'satisfaction': [],
    'max': [],
    '% satisfaction': []
}
user2numconstraints = dict(U.sum(axis=0))
for u in users:
    assigned_bnb = user2bnb[u]
    num_constraints = user2numconstraints[u]
    satisfaction = user2satisfaction[u]
    
    stats['user'].append(u)
    stats['bnb'].append(assigned_bnb)
    stats['satisfaction'].append(satisfaction)
    stats['max'].append(num_constraints)
    stats['% satisfaction'].append(satisfaction/num_constraints * 100)

print(pd.DataFrame(stats))
total_satisfaction = An[bs, us].sum()
print("total satisfaction:", total_satisfaction)
max_satisfaction = sum(user2numconstraints.values())
print("max satisfaction:", max_satisfaction)
print("total satisfaction score:", f"{round(total_satisfaction/max_satisfaction*100, 2)}%")

  user  bnb  satisfaction  max  % satisfaction
0  u_0  b_0             1    1           100.0
1  u_1  b_1             3    3           100.0
2  u_2  b_3             2    2           100.0
3  u_3  b_2             1    1           100.0
total satisfaction: 7
max satisfaction: 7
total satisfaction score: 100.0%
